In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Cropping2D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Softmax
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
import xml.etree.ElementTree as ET
import os
import cv2
import math
import pickle as pkl
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.heatmaps import HeatmapsOnImage
from tensorflow.python.client import device_lib
%matplotlib notebook
import matplotlib.pyplot as plt
import time

## Diagnostic check for GPU
Training will take far too long without a capable GPU/TPU. If you don't have such a device, you can use my colab notebook.

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

print(tf.__version__)
print(tf.test.is_gpu_available()) # True/False

# Or only check for gpu's with cuda support
print(tf.test.is_gpu_available(cuda_only=True)) 
# print(device_lib.list_local_devices())
# K.tensorflow_backend._get_available_gpus()

1 Physical GPUs, 1 Logical GPUs


## Setting Environment Variables
Point these to directory where you have stored the FIGC-CNR dataset's annotations and extracted frame images from the video files. The dictionaries are to account for the fact that some of the sequences are horizontally flipped but the coordinate systems are not, so ball data will be wrong without storing this.

In [4]:
ANNOT = "..\\FIGC\\Annotation\\Film Role-0 ID-{} T-0 m00s00-026-m00s01-020.xgtf"
IMAGE = "..\\FIGC\\Sequence{}"
TRAIN_DIR = "..\\FIGC\\TrainDataset"
MAIN_DIR = "..\\FIGC"
sequences_train = {"1":{"flip":False},"3":{"flip":False},"4":{"flip":False}}
sequences_test = {"5":{"flip": False}, "6":{"flip": True}}
TEST_DIR = "..\\FIGC\\TestDataset"

In [5]:
class Dataset:
    def __init__(self, sequences, augment=True):
        self.ns = {"base":"http://lamp.cfar.umd.edu/viper#",
                         "data":"http://lamp.cfar.umd.edu/viperdata#"}
        self.images = []
        self.imageopts = []
        self.augment = augment
        self.labels = None
        for seq,opt in sequences.items():
            curr_images = os.listdir(IMAGE.format(seq))
            seq_size = len(curr_images)
            self.images.extend([os.path.join(IMAGE.format(seq), i) for i in curr_images])
            self.imageopts.extend([opt] * seq_size)
            annotation_file = ANNOT.format(seq)  
            tree = ET.parse(annotation_file)
            
            root = tree.getroot()
            Ball = root.find(".//base:object[@name='BALL']", self.ns)
            frames = int(Ball.attrib['framespan'].split(':')[1])
            curr_labels = np.full((seq_size, 2), -1, dtype=np.int16)
            BallPos = Ball.find("./*[@name='BallPos']")
            for datapoint in BallPos.findall('data:point', self.ns):
                frameno = int(datapoint.get('framespan').split(':')[0])
                if frameno > seq_size + 7:
                    break
                x,y = int(datapoint.get('x')), int(datapoint.get('y'))
                curr_labels[frameno-1-7] = np.array([x,y])
                
            if self.labels is None:
                self.labels = curr_labels
            else:
                self.labels = np.concatenate((self.labels, curr_labels), axis=0)
                
        assert len(self.images) == self.labels.shape[0]
        assert len(self.imageopts) == self.labels.shape[0]
    
    def generate_batches(self, batch_size=16):
        set_size = len(self.images)
        image_batch = []
        map_batch = []
        for i in range(set_size):
            y_true = np.concatenate((np.zeros((68,120,1), dtype=np.float32), np.ones((68,120,1), dtype=np.float32)), axis=-1)
            img = cv2.imread(self.images[i], 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            H,W,_ = img.shape
            if H < 1088 or W < 1920:
                img = np.pad(img, ((0,1088-H),(0,1920-W),(0,0)), 'constant', constant_values=0)
            if self.imageopts[i].get("flip") == True:
                img = cv2.flip(img, 1)
                
            x,y = self.labels[i]
            visible = False if x < 0 else True
            
            x,y = int(math.floor((int(x)-1)/16)), int(math.floor((int(y)-1)/16))
            if visible:
                # Label ground truth location with ball confidence 1 and bg confidence 0
                y_true[y,x,0] = 1
                y_true[y,x,1] = 0
                # Expand background confidence to neighbours
                bg_kernel = np.ones((3,3), dtype=np.uint8)
                y_true[:,:,1] = cv2.erode(y_true[:,:,1], bg_kernel, iterations=1)
            
            # Input images being resized to 480 by 272 to improve performance
            image_batch.append(cv2.resize(img, dsize=(480,272)))
            map_batch.append(y_true)
            if len(image_batch) == batch_size:
                yield (np.asarray(image_batch), np.asarray(map_batch))
                
                if self.augment:
                    # Image Augmentation 
                    seq = iaa.Sequential([
                        iaa.OneOf([
                            iaa.MultiplyBrightness((0.5,1.5)),
                            iaa.MultiplyHue((0.5,1.5)),
                            iaa.MultiplySaturation((0.5,1.5)),
                            iaa.LinearContrast((0.4,1.6))
                        ]),
                        iaa.Fliplr(0.5),
                        iaa.Sometimes(0.50, iaa.Crop(percent=(0.0,0.25)))
                    ])
                    images_aug, maps_aug = seq(images=image_batch, heatmaps=map_batch)

                    yield (np.asarray(images_aug), np.asarray(maps_aug))
                
                image_batch.clear()
                map_batch.clear()
        if image_batch:
            yield (np.asarray(image_batch), np.asarray(map_batch))
            
            if self.augment:
                # Image Augmentation 
                seq = iaa.Sequential([
                        iaa.OneOf([
                            iaa.MultiplyBrightness((0.5,1.5)),
                            iaa.MultiplyHue((0.5,1.5)),
                            iaa.MultiplySaturation((0.5,1.5)),
                            iaa.LinearContrast((0.4,1.6))
                        ]),
                        iaa.Fliplr(0.5),
                        iaa.Sometimes(0.50, iaa.Crop(percent=(0.0,0.33)))
                    ])
                images_aug, maps_aug = seq(images=image_batch, heatmaps=map_batch)

                yield (np.asarray(images_aug), np.asarray(maps_aug))
            
            del image_batch, map_batch
            
def load(DIR):
    def gen_dataset():
        file_list = os.listdir(DIR)
        # Infinitely iterable as Keras requires
        while True:
            for file in file_list:
                with open(os.path.join(DIR, file), 'rb') as f:
                    x_train, y_train = pkl.load(f)
                    yield (x_train, y_train)
    return gen_dataset

In [6]:
def deepball_loss_function(y_true, y_pred):
    # y_true (batch_size, 68, 120, 2)
    # y_pred (batch_size, 68, 120, 2)
    
    ball_gt, bg_gt = y_true[:,:,:,0], y_true[:,:,:,1]
    N = K.sum(ball_gt, axis=(1,2)) + 1
    M = K.sum(bg_gt, axis=(1,2)) + 1
    zer = K.zeros_like(ball_gt)
    
    y_pred = K.log(y_pred)
    ball_cm = y_pred[:,:,:,0]
    bg_cm = y_pred[:,:,:,1]
    
    Lpos = K.sum(zer + (ball_cm * ball_gt), axis=(1,2))
    Lpos = K.sum(K.zeros_like(N) + (Lpos / tf.maximum(1.0, N)))
    
    Lneg = K.sum(zer + (bg_cm * bg_gt), axis=(1,2))
    Lneg = K.sum(K.zeros_like(M) + (Lneg / tf.maximum(1.0, M)))
#     print(K.eval(Lpos),K.eval(Lneg))

    # Multiplying by batch_size as Keras automatically averages the scalar output over it
    return (-Lpos - 0.3*Lneg) * 16
    

In [7]:
def deepball_precision(y_true, y_pred):
    ball_gt = y_true[:,:,:,0]
    ball_cm = y_pred[:,:,:,0]
    
    thre_ball_cm = K.cast(K.greater(ball_cm, 0.998), "float32")
    tp = K.sum(ball_gt * thre_ball_cm)
    totalp = K.sum(K.max(thre_ball_cm, axis=(1,2)))
    
    return tp/tf.maximum(1.0, totalp)  

In [8]:
def peak_tp_confidence(y_true, y_pred):
    ball_gt = y_true[:,:,:,0]
    ball_cm = y_pred[:,:,:,0]
    N = K.sum(ball_gt, axis=(1,2))
    num_tp = K.sum(N)
    peak_c = K.max(ball_cm, axis=(1,2))
    
    return K.sum((N * peak_c)) / tf.maximum(1.0, num_tp)

In [9]:
def peak_fp_confidence(y_true, y_pred):
    ball_gt = y_true[:,:,:,0]
    ball_cm = y_pred[:,:,:,0]
    N = 1 - K.sum(ball_gt, axis=(1,2))
    num_fp = K.sum(N)
    peak_c = K.max(ball_cm, axis=(1,2))
    
    return K.sum((N * peak_c)) / tf.maximum(1.0, num_fp)

In [12]:
# Functional API
def _conv_block(inp, convs, pool=True):
    x = inp
    count = 0
    for conv in convs:
        count += 1
        x = Conv2D(conv['filter'],
                   conv['kernel'],
                   strides=conv['stride'],
                   padding='same',
                   use_bias=False)(x)
        x = BatchNormalization(epsilon=0.001)(x)
        x = ReLU()(x)
    if pool:
        x = MaxPool2D(pool_size=2)(x)
    return x

def get_functional_deepball_model():
    #Input image
    input_image = Input(shape=(272, 480, 3))

    # Conv1
    x = _conv_block(input_image, [{'filter': 8, 'kernel': 7, 'stride': 2, 'layer_idx': 0},
                                  {'filter': 8, 'kernel': 3, 'stride': 1, 'layer_idx': 1}])
    conv1 = x
    # Conv2
    x = _conv_block(x, [{'filter': 16, 'kernel': 3, 'stride': 1, 'layer_idx': 2},
                        {'filter': 16, 'kernel': 3, 'stride': 1, 'layer_idx': 3}])
    conv2 = x

    # Conv3
    x = _conv_block(x, [{'filter': 32, 'kernel': 3, 'stride': 1, 'layer_idx': 4},
                        {'filter': 32, 'kernel': 3, 'stride': 1, 'layer_idx': 5}])
    conv3 = x

    # Upsampling Conv2
    upsampled_conv2 = UpSampling2D(size=(2,2))(conv2)

    # Upsampling Conv3
    upsampled_conv3 = UpSampling2D(size=(4,4))(conv3)

    # Concatenation along channels axis
    concat = Concatenate(axis=-1)([conv1, upsampled_conv2, upsampled_conv3])
    x = concat

    # Conv4
    x = _conv_block(x, [{'filter': 56, 'kernel': 3, 'stride': 1, 'layer_idx':6},
                        {'filter': 2, 'kernel': 3, 'stride': 1, 'layer_idx': 7}], pool=False)
    
#     conv3c = Reshape((32*30*17,))(conv3)
#     cout = Dense(1, activation='sigmoid')(Dense(200, activation='relu')(conv3c))
    
    x = Softmax(axis=-1)(x)
    model = keras.Model(inputs=input_image, outputs=x)
    
    return model

In [14]:
def create_batches(dataset, DIR):
    d_generator = dataset.generate_batches(batch_size=16)
    batchno = 1
    for x_train, y_train in d_generator:
        with open(os.path.join(DIR, '{}.pkl'.format(batchno)), 'wb') as f:
            pkl.dump([x_train, y_train], f)
        print('saved to disk: {}'.format(batchno))
        batchno+=1

def get_compiled_deepball_model():
    # define the model
    model = get_functional_deepball_model()
#     model.summary()
    model.compile(optimizer='adam', loss=deepball_loss_function, 
                  metrics=[deepball_precision])
    return model

def train(model):
    train_dataset = tf.data.Dataset.from_generator(load(TRAIN_DIR),
                                                    (tf.uint8, tf.uint8), 
                                                    (tf.TensorShape([None,272,480,3]),tf.TensorShape([None,68,120,2]))).unbatch().map(lambda *args: [tf.cast(elem, tf.float32) for elem in args]).batch(16)
    model.fit(train_dataset, epochs=75, verbose=2, steps_per_epoch=1126)

    model.save_weights('deepball_weights.h5')
    keras.models.save_model(model, "deepballlocal.h5", save_format="h5")

## Batching and Training
The create_batches function will pickle batches of 16 samples (image,target_map) in the TrainDataset directory.

In [16]:
# train_dataset = Dataset(sequences_train)
# create_batches(train_dataset, TRAIN_DIR)

In [17]:
# model = get_compiled_deepball_model()
# train(model)

In [19]:
# test_dataset = Dataset(sequences_test, augment=False)
# create_batches(test_dataset, TEST_DIR)

In [20]:
customObj = {'deepball_loss_function': deepball_loss_function, 'deepball_precision': deepball_precision}
testmodel = keras.models.load_model('./deepballlocal.h5', custom_objects=customObj)
# evaluation = testmodel.evaluate_generator(test_load, steps=375, verbose=1)
# print(evaluation)

In [21]:
def testNet(model):
    sampimg = cv2.imread(os.path.join(MAIN_DIR, "Sequence6", "0010.bmp"), 1)
    sampimg = cv2.cvtColor(sampimg, cv2.COLOR_BGR2RGB)
    H,W,_ = sampimg.shape

    cmap = model.predict(np.array([cv2.resize(sampimg, (480,272))]), batch_size=1, verbose=1)
#     cmap = K.exp(cmap)

    ball_cm = cmap[0,:,:,0]
    bg_cm = cmap[0,:,:,1]
    
    pos = np.unravel_index(np.argmax(ball_cm, axis=None), ball_cm.shape)
    yp, xp = pos
    print(xp, yp)
    yp, xp = int(math.floor(16*yp)), int(math.floor(16*xp))
    print(xp, yp)
    print('Peak ball C : {}'.format(ball_cm[pos]))
    plt.figure(figsize=(8,10))
    plt.subplot(3,1,1)
    plt.imshow(bg_cm, cmap='gray')
    plt.subplot(3,1,2)
    plt.imshow(ball_cm, cmap='gray')
    plt.subplot(3,1,3)
    cv2.circle(sampimg, (xp,yp), 16, (43,0,255), thickness=2)
    plt.imshow(sampimg)
    plt.show()

In [22]:
# testNet(testmodel)

In [23]:
def display_ball_box(image, x, y):
    if x < 0:
        print("*** No ball detected ***")
    else:
        ball_pos = np.array([y,x])
        cv2.circle(image, (x, y), 16, (255,0,0), thickness=2)
    return image

In [27]:
THRESHOLD_VAL = 0.999999 
def ballTrack(cap, output, model):
    trackers = []
    i = 0
    frame_rate_divider = 1
#     bpos = deque()
    while(cap.isOpened()):
        stime = time.time()
        ret, frame = cap.read()
        if ret:
            if i % frame_rate_divider == 0:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                (h, w) = frame.shape[:2]
                ky, kx = 4 * h/272.0, 4 * w/480.0
                cmap = model.predict(np.array([cv2.resize(frame.astype(np.float32), (480,272))]), batch_size=1, verbose=1)
                cm = cmap[0,:,:,0]
                pos = np.unravel_index(np.argmax(cm, axis=None), cm.shape)
                y,x = pos
                print(cm[y,x])
                x = -1 if cm[y,x] < THRESHOLD_VAL else x
                y,x = math.floor(ky * y), math.floor(kx * x)
                
                frame = display_ball_box(frame, x, y)
#                 bpos.append((y,x))
                output.write(frame)
                cv2.imshow('frame', frame)
                i += 1
                
            print('FPS {:.1f}'.format(1 / (time.time() - stime)))

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    output.release()
    cv2.destroyAllWindows()

## Set input stream to video file

In [28]:
cap = cv2.VideoCapture('Seq6.avi')
size = (
	int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
	int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
codec = cv2.VideoWriter_fourcc(*'DIVX')
output = cv2.VideoWriter('Seq6Ballco4.avi',codec,25.0,size)
# outputSeg = cv2.VideoWriter('Seq6Seg.avi', codec,25.0,size)

In [29]:
ballTrack(cap,output,testmodel)

1/1 [==============================] - 0s 10ms/sample
0.9999404
*** No ball detected ***
FPS 7.3
1/1 [==============================] - 0s 7ms/sample
0.9999552
*** No ball detected ***
FPS 15.4
1/1 [==============================] - 0s 7ms/sample
0.9999546
*** No ball detected ***
FPS 15.6
1/1 [==============================] - 0s 7ms/sample
0.99997497
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.99997973
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.9999753
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.9999795
*** No ball detected ***
FPS 16.9
1/1 [==============================] - 0s 6ms/sample
0.9999788
*** No ball detected ***
FPS 17.9
1/1 [==============================] - 0s 6ms/sample
0.9999782
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.99997723
*** No ball detected ***
FPS 16.7
1/1 [======================

0.9999254
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 7ms/sample
0.999913
*** No ball detected ***
FPS 15.1
1/1 [==============================] - 0s 6ms/sample
0.99992275
*** No ball detected ***
FPS 14.3
1/1 [==============================] - 0s 6ms/sample
0.999882
*** No ball detected ***
FPS 15.6
1/1 [==============================] - 0s 7ms/sample
0.99989593
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 7ms/sample
0.9999057
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.999863
*** No ball detected ***
FPS 2.5
1/1 [==============================] - 0s 7ms/sample
0.9998423
*** No ball detected ***
FPS 15.4
1/1 [==============================] - 0s 6ms/sample
0.99985564
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.99984276
*** No ball detected ***
FPS 15.1
1/1 [==============================] - 0s 7ms/sample
0.9998946
*** No ball detected

1/1 [==============================] - 0s 7ms/sample
0.99994385
*** No ball detected ***
FPS 15.6
1/1 [==============================] - 0s 6ms/sample
0.9999589
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.99996436
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 7ms/sample
0.9999627
*** No ball detected ***
FPS 15.6
1/1 [==============================] - 0s 6ms/sample
0.9999511
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 7ms/sample
0.9999448
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.99994326
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.99992764
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.9999231
*** No ball detected ***
FPS 16.9
1/1 [==============================] - 0s 6ms/sample
0.99994576
*** No ball detected ***
FPS 16.4
1/1 [====================

0.9998166
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.9998041
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9998473
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 7ms/sample
0.99987113
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 7ms/sample
0.99986744
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 7ms/sample
0.9998093
*** No ball detected ***
FPS 14.5
1/1 [==============================] - 0s 7ms/sample
0.99980575
*** No ball detected ***
FPS 14.7
1/1 [==============================] - 0s 6ms/sample
0.99982387
*** No ball detected ***
FPS 15.4
1/1 [==============================] - 0s 6ms/sample
0.9998492
*** No ball detected ***
FPS 14.5
1/1 [==============================] - 0s 6ms/sample
0.9998542
*** No ball detected ***
FPS 14.9
1/1 [==============================] - 0s 7ms/sample
0.99987257
*** No ball det

1/1 [==============================] - 0s 7ms/sample
0.9999368
*** No ball detected ***
FPS 15.4
1/1 [==============================] - 0s 5ms/sample
0.99992573
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.99993753
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9999448
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.9999305
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.9998952
*** No ball detected ***
FPS 15.6
1/1 [==============================] - 0s 6ms/sample
0.99994135
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.99987876
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.999866
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.9998387
*** No ball detected ***
FPS 16.4
1/1 [======================

...

1/1 [==============================] - 0s 5ms/sample
0.9999547
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.99994564
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 7ms/sample
0.99993753
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.99993455
*** No ball detected ***
FPS 17.2
1/1 [==============================] - 0s 6ms/sample
0.99994373
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.9999397
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.99994373
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.999962
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.9999864
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9999769
*** No ball detected ***
FPS 14.9
1/1 [=====================

0.99998164
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.99998295
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.9999856
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.99998355
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.9999883
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9999863
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9999887
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.9999815
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.9999857
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 7ms/sample
0.99998546
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 7ms/sample
0.9999838
*** No ball dete

1/1 [==============================] - 0s 6ms/sample
0.99998295
*** No ball detected ***
FPS 16.9
1/1 [==============================] - 0s 7ms/sample
0.99998546
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 7ms/sample
0.99998236
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 7ms/sample
0.99997556
*** No ball detected ***
FPS 14.9
1/1 [==============================] - 0s 7ms/sample
0.9999815
*** No ball detected ***
FPS 15.4
1/1 [==============================] - 0s 6ms/sample
0.99997973
*** No ball detected ***
FPS 15.6
1/1 [==============================] - 0s 6ms/sample
0.9999796
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.9999819
*** No ball detected ***
FPS 15.1
1/1 [==============================] - 0s 6ms/sample
0.99998415
*** No ball detected ***
FPS 15.6
1/1 [==============================] - 0s 7ms/sample
0.9999801
*** No ball detected ***
FPS 15.9
1/1 [===================

0.99998033
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.999977
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.99997854
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.99997914
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.99998033
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9999783
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.999979
*** No ball detected ***
FPS 16.1
1/1 [==============================] - 0s 6ms/sample
0.99997723
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.9999782
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 8ms/sample
0.9999801
*** No ball detected ***
FPS 14.3
1/1 [==============================] - 0s 6ms/sample
0.9999778
*** No ball detec

1/1 [==============================] - 0s 6ms/sample
0.9999927
*** No ball detected ***
FPS 15.9
1/1 [==============================] - 0s 6ms/sample
0.9999968
*** No ball detected ***
FPS 17.2
1/1 [==============================] - 0s 6ms/sample
0.99999523
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9999949
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9999939
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.9999939
*** No ball detected ***
FPS 16.7
1/1 [==============================] - 0s 6ms/sample
0.99999416
*** No ball detected ***
FPS 16.9
1/1 [==============================] - 0s 6ms/sample
0.99999547
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 7ms/sample
0.9999938
*** No ball detected ***
FPS 16.4
1/1 [==============================] - 0s 6ms/sample
0.99999404
*** No ball detected ***
FPS 16.7
1/1 [=====================